In [1]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path
import requests


In [2]:
data_file=Path("Resources/global-shark-attack-raw.xlsx")
rcPrompt=''
if Path(data_file).is_file():
    fileFix = True
    while fileFix:
        try:
            data_file.unlink()
            break
        except PermissionError:
            print('Unable to delete a previously created raw data file because it is currently in use by another process.\nIf you do not get a new file, data may be out of date.\n\n"' + str(data_file) + '"')
            while True:
                rcPrompt = input('Do you want to [c]ontinue or [r]etry?').upper()
                if rcPrompt not in ['R','C']:
                    print('Invalid reponse.')
                else:
                    if rcPrompt == 'C':
                        fileFix = False
                        break
                    else:
                        print('')
                        break

if rcPrompt!='C':
    download = requests.get("https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/global-shark-attack/exports/xlsx?lang=en&use_labels=true")
    savefile = open(data_file,'xb')
    savefile.write(download.content)
    savefile.close()
else:
    print('Unable to download the raw data. Data may be outdated.')


In [3]:
#read the excel file into a dataframe
all_data_df = pd.read_excel(data_file)

In [4]:
#Clean up items that may not be shark attacks
clean_sharks_df = all_data_df.dropna(subset=['Type'])
clean_sharks_df = clean_sharks_df.dropna(subset=['Year','Date'], how='all')
clean_sharks_df = clean_sharks_df.loc[(~clean_sharks_df['Type'].isin(['?','Invalid','Questionable','Unconfirmed','Under investigation','Unverified'])),:]

In [5]:
clean_sharks_df.to_excel(Path("Resources/global-shark-attack_cleaned.xlsx"), index=False)